In [13]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None, 'display.max_rows', 100)

atp_data = pd.read_csv('atp_data_clean.csv')
stats_data = pd.read_csv('ATP2000_2021.csv')
players = pd.read_csv('atp_players.csv', header=None)

atp_data = atp_data.iloc[:,1:]
atp_data.rename({'J1' : 'Winner', 'J2': 'Loser'}, axis=1, inplace=True)
stats_data = stats_data.iloc[:,1:]

In [14]:
#Funtion to convert player names we have in the file atp_players (in which we have the split between first name and last name) in the format of atp_data and to put every letter in uppercase
def format_name(first_name, last_name):
  formatted_name = last_name + ' '
  first_name_split = first_name.split(' ')
  if first_name_split != ['']:
    for name in first_name_split:
      if name !='':
        formatted_name += name[0] + '.'
  return formatted_name.upper()

#formatting players names in atp_players
players = players.dropna(axis=0, subset=[2])
players[1] = players[1].fillna('')
players['formatted_name'] = players.apply(lambda row: format_name(row[1],row[2]), axis=1)
players = players.rename({0 : 'id'}, axis=1)
players.drop([1,2,3,4,5], axis=1, inplace=True)

In [15]:
#We also convert names in atp_data in uppercase to avoid capitalisation issues
atp_data['Winner'] = atp_data.apply(lambda row: row['Winner'].upper(), axis=1)
atp_data['Loser'] = atp_data.apply(lambda row: row['Loser'].upper(), axis=1)

In [16]:
#We merge stats_data file with player names on their id
#For the winners
stats_data_formatted_name = pd.merge(left=stats_data, right=players, left_on='winner_id', right_on='id', how = 'left')
#Then for the losers
stats_data_formatted_name = pd.merge(left=stats_data_formatted_name, right=players, left_on='loser_id', right_on='id', how = 'left')
#And we delete id_x (winner) and id_y (loser) that we just add
stats_data_formatted_name.drop(['id_x', 'id_y'], axis=1, inplace=True)

#We notice that stats_data_formatted_name does not contain any name with '-'
stats_data_formatted_name[(stats_data_formatted_name['formatted_name_x'].str.contains('-')) | (stats_data_formatted_name['formatted_name_y'].str.contains('-'))]
#so we replace the '-' in atp_data names by ' '
atp_data['Winner'] = atp_data.apply(lambda row: row['Winner'].replace('-', ' '), axis=1)
atp_data['Loser'] = atp_data.apply(lambda row: row['Loser'].replace('-', ' '), axis=1)

#Some names are slightly different
atp_data[['Winner', 'Loser']] = atp_data[['Winner', 'Loser']].replace({'DOLGOPOLOV O.' : 'DOLGOPOLOV A.'})
atp_data[['Winner', 'Loser']] = atp_data[['Winner', 'Loser']].replace({'BAUTISTA R.' : 'BAUTISTA AGUT R.'})
atp_data[['Winner', 'Loser']] = atp_data[['Winner', 'Loser']].replace({'KUZNETSOV AN.' : 'KUZNETSOV A.'})
atp_data[['Winner', 'Loser']] = atp_data[['Winner', 'Loser']].replace({'RAMOS A.' : 'RAMOS VINOLAS A.'})
stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']] = stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']].replace({'RAMOS A.' : 'RAMOS VINOLAS A.'})
stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']] = stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']].replace({'VASSELIN E.R.' : 'ROGER VASSELIN E.'})
stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']] = stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']].replace({'BOGOMOLOV JR A.' : 'BOGOMOLOV A.'})
stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']] = stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']].replace({'SCHWARTZMAN D.S.' : 'SCHWARTZMAN D.'})
stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']] = stats_data_formatted_name[['formatted_name_x', 'formatted_name_y']].replace({'ESTRELLA V.' : 'ESTRELLA BURGOS V.'})

In [17]:
#Formatting of the match dates
#In atp_data
atp_data['Date'] = pd.to_datetime(atp_data['Date'])
#In stats_data
stats_data_formatted_name['tourney_date'] = pd.to_datetime(stats_data_formatted_name['tourney_date'], format='%Y%m%d')
#stats_data_formatted_name.drop(stats_data[stats_data_formatted_name['tourney_date'] > datetime(year=2018, month=3, day=4)].index, axis=0, inplace=True)
stats_data_formatted_name.sort_values(by='tourney_date', inplace=True)

In [18]:
#Merging of atp_data and stats_data on date and winner and loser names
stats_data_formatted_name.rename({'tourney_date' : 'Date', 'formatted_name_x' : 'Winner', 'formatted_name_y' : 'Loser'}, axis=1, inplace=True)
#atp_data does not contain Davis Cup matches so we delete them from stats_data_formatted_name
atp_data[atp_data['Tournament'].str.contains('Davis')]
stats_data_formatted_name.drop(stats_data_formatted_name[stats_data_formatted_name['tourney_name'].str.contains('Davis Cup')].index, axis=0, inplace=True)
#we have to merge_asof as some dates have a 1-day discrepancy probably due to time zone. But we do have to put the tolerance parameter to 16 (max duration of a tournament) as some tournaments are filled with a single date
data_merged = pd.merge_asof(right=stats_data_formatted_name, left=atp_data, on=['Date'], by=['Winner', 'Loser'], tolerance=timedelta(days=16), direction='nearest')

In [19]:
#Deleting matches that are not in atp_data (qualificative rounds ER and 'ATP Next Gen Finals' BR)
data_merged.drop(data_merged[(data_merged['round']=='ER') | (data_merged['round']=='BR')].index, axis=0, inplace=True)

In [20]:
data_merged.isna().sum()

ATP                       0
Location                  0
Tournament                0
Date                      0
Series                    0
Court                     0
Surface                   0
Round                     0
Best of                   0
Winner                    0
Loser                     0
WRank                     0
LRank                     0
Wsets                     0
Lsets                     0
Comment                   0
PSW                       0
PSL                       0
B365W                     0
B365L                     0
elo_winner                0
elo_loser                 0
proba_elo                 0
DeltaW                    0
DeltaL                    0
tourney_id             2354
tourney_name           2354
surface                2354
draw_size              2354
tourney_level          2354
match_num              2354
winner_id              2354
winner_seed            2354
winner_entry           2354
winner_name            2354
winner_hand         

In [21]:
#Replacing NA from 'minutes' played with 3 sets and 5 sets matches medians
[median_minutes_3sets, median_minutes_5sets] = data_merged[['Best of', 'minutes']].groupby('Best of').median().loc[:,'minutes']
data_merged['minutes'] = np.where((data_merged['Best of']==3) & (data_merged['minutes'].isna()), median_minutes_3sets, data_merged['minutes'])
data_merged['minutes'] = np.where((data_merged['Best of']==5) & (data_merged['minutes'].isna()), median_minutes_5sets, data_merged['minutes'])

In [22]:
#Deleting redundant features 
data_merged.drop(['ATP', 'Location', 'Series', 'Round', 'Best of', 'tourney_name', 'surface', 'draw_size', 'match_num', 'winner_name', 'winner_ioc', 'loser_name', 'loser_ioc', 'best_of', 'winner_rank', 'loser_rank', 'tourney_date_proper'], axis=1, inplace=True)

In [23]:
#We drop the remaining NAs from the merge
data_merged.drop(data_merged[data_merged['tourney_id'].isna()].index, axis=0, inplace=True)

In [24]:
data_merged.isna().sum()

Tournament               0
Date                     0
Court                    0
Surface                  0
Winner                   0
Loser                    0
WRank                    0
LRank                    0
Wsets                    0
Lsets                    0
Comment                  0
PSW                      0
PSL                      0
B365W                    0
B365L                    0
elo_winner               0
elo_loser                0
proba_elo                0
DeltaW                   0
DeltaL                   0
tourney_id               0
tourney_level            0
winner_id                0
winner_seed              0
winner_entry             0
winner_hand              0
winner_ht             1112
winner_age               0
loser_id                 0
loser_seed               0
loser_entry              0
loser_hand               0
loser_ht              2310
loser_age                0
score                    0
round                    0
minutes                  0
w

In [25]:
#We add manually some missing heights
data_merged.loc[data_merged['Winner']=='CORIC B.', 'winner_ht'] = 188
data_merged.loc[data_merged['Loser']=='CORIC B.', 'loser_ht'] = 188

data_merged.loc[data_merged['Winner']=='VESELY J.', 'winner_ht'] = 198
data_merged.loc[data_merged['Loser']=='VESELY J.', 'loser_ht'] = 198

data_merged.loc[data_merged['Winner']=='EDMUND K.', 'winner_ht'] = 188
data_merged.loc[data_merged['Loser']=='EDMUND K.', 'loser_ht'] = 188

data_merged.loc[data_merged['Winner']=='STRUFF J.L.', 'winner_ht'] = 193
data_merged.loc[data_merged['Loser']=='STRUFF J.L.', 'loser_ht'] = 193

data_merged.loc[data_merged['Winner']=='KHACHANOV K.', 'winner_ht'] = 198
data_merged.loc[data_merged['Loser']=='KHACHANOV K.', 'loser_ht'] = 198

data_merged.loc[data_merged['Winner']=='CHUNG H.', 'winner_ht'] = 188
data_merged.loc[data_merged['Loser']=='CHUNG H.', 'loser_ht'] = 188

data_merged.loc[data_merged['Winner']=='DONALDSON J.', 'winner_ht'] = 188
data_merged.loc[data_merged['Loser']=='DONALDSON J.', 'loser_ht'] = 188

data_merged.loc[data_merged['Winner']=='RUBLEV A.', 'winner_ht'] = 188
data_merged.loc[data_merged['Loser']=='RUBLEV A.', 'loser_ht'] = 188

data_merged.loc[data_merged['Winner']=='MEDVEDEV D.', 'winner_ht'] = 198
data_merged.loc[data_merged['Loser']=='MEDVEDEV D.', 'loser_ht'] = 198

data_merged.loc[data_merged['Winner']=='GROTH S.', 'winner_ht'] = 193
data_merged.loc[data_merged['Loser']=='GROTH S.', 'loser_ht'] = 193

data_merged.loc[data_merged['Winner']=='MONTEIRO T.', 'winner_ht'] = 183
data_merged.loc[data_merged['Loser']=='MONTEIRO T.', 'loser_ht'] = 183

data_merged.loc[data_merged['Winner']=='BEMELMANS R.', 'winner_ht'] = 183
data_merged.loc[data_merged['Loser']=='BEMELMANS R.', 'loser_ht'] = 183

data_merged.loc[data_merged['Winner']=='ALBOT R.', 'winner_ht'] = 175
data_merged.loc[data_merged['Loser']=='ALBOT R.', 'loser_ht'] = 175

data_merged.loc[data_merged['Winner']=='DANIEL T.', 'winner_ht'] = 191
data_merged.loc[data_merged['Loser']=='DANIEL T.', 'loser_ht'] = 191

data_merged.loc[data_merged['Winner']=='BAGNIS F.', 'winner_ht'] = 183
data_merged.loc[data_merged['Loser']=='BAGNIS F.', 'loser_ht'] = 183

data_merged.loc[data_merged['Winner']=='FABBIANO T.', 'winner_ht'] = 173
data_merged.loc[data_merged['Loser']=='FABBIANO T.', 'loser_ht'] = 173

data_merged.loc[data_merged['Winner']=='TIAFOE F.', 'winner_ht'] = 188
data_merged.loc[data_merged['Loser']=='TIAFOE F.', 'loser_ht'] = 188

data_merged.loc[data_merged['Winner']=='NISHIOKA Y.', 'winner_ht'] = 170
data_merged.loc[data_merged['Loser']=='NISHIOKA Y.', 'loser_ht'] = 170

data_merged.loc[data_merged['Winner']=='OLIVO R.', 'winner_ht'] = 178
data_merged.loc[data_merged['Loser']=='OLIVO R.', 'loser_ht'] = 178

data_merged.loc[data_merged['Winner']=='MUNOZ DE LA NAVA D.', 'winner_ht'] = 175
data_merged.loc[data_merged['Loser']=='MUNOZ DE LA NAVA D.', 'loser_ht'] = 175

data_merged.loc[data_merged['Winner']=='CECCHINATO M.', 'winner_ht'] = 185
data_merged.loc[data_merged['Loser']=='CECCHINATO M.', 'loser_ht'] = 185

data_merged.loc[data_merged['Winner']=='KOKKINAKIS T.', 'winner_ht'] = 193
data_merged.loc[data_merged['Loser']=='KOKKINAKIS T.', 'loser_ht'] = 193

data_merged.loc[data_merged['Winner']=='MELZER G.', 'winner_ht'] = 188
data_merged.loc[data_merged['Loser']=='MELZER G.', 'loser_ht'] = 188

data_merged.loc[data_merged['Winner']=='THOMPSON J.', 'winner_ht'] = 183
data_merged.loc[data_merged['Loser']=='THOMPSON J.', 'loser_ht'] = 183

data_merged.loc[data_merged['Winner']=='SHAPOVALOV D.', 'winner_ht'] = 185
data_merged.loc[data_merged['Loser']=='SHAPOVALOV D.', 'loser_ht'] = 185

In [26]:
data_merged.isna().sum()

Tournament               0
Date                     0
Court                    0
Surface                  0
Winner                   0
Loser                    0
WRank                    0
LRank                    0
Wsets                    0
Lsets                    0
Comment                  0
PSW                      0
PSL                      0
B365W                    0
B365L                    0
elo_winner               0
elo_loser                0
proba_elo                0
DeltaW                   0
DeltaL                   0
tourney_id               0
tourney_level            0
winner_id                0
winner_seed              0
winner_entry             0
winner_hand              0
winner_ht              453
winner_age               0
loser_id                 0
loser_seed               0
loser_entry              0
loser_hand               0
loser_ht              1364
loser_age                0
score                    0
round                    0
minutes                  0
w

In [27]:
#We fill the heights still missing by the median in order to keep all the playes (as we'll see later the heights doesn't really impact the models)
data_merged["winner_ht"] = data_merged["winner_ht"] .fillna(data_merged["winner_ht"].median())
data_merged["loser_ht"] = data_merged["loser_ht"] .fillna(data_merged["loser_ht"].median())

In [28]:
data_merged.isna().sum()

Tournament              0
Date                    0
Court                   0
Surface                 0
Winner                  0
Loser                   0
WRank                   0
LRank                   0
Wsets                   0
Lsets                   0
Comment                 0
PSW                     0
PSL                     0
B365W                   0
B365L                   0
elo_winner              0
elo_loser               0
proba_elo               0
DeltaW                  0
DeltaL                  0
tourney_id              0
tourney_level           0
winner_id               0
winner_seed             0
winner_entry            0
winner_hand             0
winner_ht               0
winner_age              0
loser_id                0
loser_seed              0
loser_entry             0
loser_hand              0
loser_ht                0
loser_age               0
score                   0
round                   0
minutes                 0
w_ace                 179
w_df        

In [29]:
data_merged.to_csv('ATP_merged_clean.csv', sep=',')